In [1]:
import emoji
import re
import random
import pandas as pd 
import numpy as np
from pandarallel import pandarallel
from opencc import OpenCC
from tqdm.auto import tqdm

In [2]:
df_starbux = pd.read_pickle("./data_starbucks.pkl")[["docid", 'headline_content', 'product_gt', 'brand_gt']]
df_starbux.rename(columns={'headline_content': "context", 'product_gt': 'product_gpt', 'brand_gt': 'brand_gpt'}, inplace=True)
df_starbux.drop_duplicates(subset=['docid'], inplace=True)
df_starbux.reset_index(drop=True, inplace=True)
df_starbux

,docid,context,product_gpt,brand_gpt
0,bcf76d5a80f713fc1b28ceb62accffef,"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[]
1,bd003ce6a01a0fd0627f443a5cd9635a,别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[]
2,bd1ac8719255e993aa5ff2198c81c5aa,//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000D_#莫...,[抹茶星冰乐],[]
3,bd468468ea8a82bba99a718531c3b039,[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000D_打工人小葛喝...,[太妃榛果拿铁],[]
4,100fd688029bedd1b42de4ba13b0e9fd,[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000D_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克]
...,...,...,...,...
157846,9e946afadbdeeda57e32b28af16c1472,"#京奇好生活 #霸王茶姬 带你走进茶的王国,霸王茶姬#京奇好生活 #霸王茶姬 带你走进茶的王...",[],[]
157847,194caedc8b299d884d3ed456b045a1f8,霸王茶姬2r #霸王茶姬#霸王茶姬以茶会友#点缀生活的鲜花,[],[霸王茶姬]
157848,b8e98ed9585f8a12f076fd8269cf0936,不吃晚饭减肥day1 韩式牛肉拌饭 一罐可乐 两杯霸王茶姬 几颗阳光玫瑰和蓝莓 (起晚了直接...,[],[霸王茶姬]
157849,8b307513263a112be1eb85e52d704515,"一些谷 很美丽的fukuya和很美丽的亚克力们,实在无法立牌! 本想把辛的这个立牌塞进霸王茶...",[],[霸王茶姬]


In [3]:
df_yht = pd.read_pickle("./data_yihetang.pkl")[["doc_id", 'split_content', 'product_gpt', 'brand_gpt']]
df_yht.rename(columns={'doc_id': 'docid', 'split_content': 'context'}, inplace=True)
df_yht

,docid,context,product_gpt,brand_gpt
0,2024013121789827482_0,你还不知道吗？益禾堂和猪猪侠联名了，益禾堂的草莓麻薯多多和草莓麻薯抹茶，香甜好喝，宝子们，赶...,"[草莓麻薯多多, 草莓麻薯抹茶, 奶茶]","[益禾堂, 猪猪侠]"
1,2024011721773863439_0,"薅羊毛攻略(1)-免费奶茶\n社群分享,羊毛不漏过\n加入各类省钱群,交流心得,分享优惠信息...",[奶茶],"[茶百道, 大众点评]"
2,202401177133469_0,【2024温江大庙会·1月27日启幕】¥29.9期抢单人特惠夜场票~非遗大秀·打铁花+新春游...,"[抢单人特惠夜场票, 新春游园会, 东方幻想大巡游, 祈福放河灯, 赏花灯, 庙会游园, 演...","[国色天乡陆地乐园, 温江大庙会, CY2024]"
3,2024012921786349177_0,#欢欢喜喜过大年 #年味 #新年倒计时 #演出现场 #举杯益禾堂祥龙十八涨 @益禾堂\n#欢...,[祥龙十八涨],[益禾堂]
4,2024011521777048185_0,#我是爆单王 #益禾堂随心选 #一杯饮品的快乐\n#我是爆单王 #益禾堂随心选 #一杯饮品的快乐,[随心选饮品],[益禾堂]
...,...,...,...,...
51525,2024010721779555610_1,"必须是新题材,新事物。\n周四我深水重仓中文在线,周五量化砸盘转盈为亏。在目前这个节点而言,...",[],"[中文在线, 鸿蒙, 东方中科, 亚华电子, 延华智能, 洲明科技, 光伏]"
51526,2024010721779693285_10,"年初至今,SW社会服务板块下跌1.14%,沪深300下跌2.97%,社会服务板块跑赢大盘1....","[70架飞机, 机场贵宾室, 免费机上Wi-Fi]","[SW社会服务, SW酒店餐饮, SW旅游及景区, SW教育, SW专业服务, 长白山, 大..."
51527,2024010721779853956_0,【01.07.解禁】美指小幅收涨！央行定调十项重点工作！国常会！数字经济促进共同富裕！证监会...,[],[]
51528,202401173863473_0,港股开盘 | 恒指低开0.64% 造车新势力、科网股表现较弱\n1月17日，恒生指数低开0....,"[恒生指数, 国企指数, 恒生科技指数, 道琼斯指数, 标普500指数, 纳斯达克指数, 美...","[蔚来, 小鹏汽车, 波音, AMD, 英伟达]"


In [4]:
df_yht['docid'].is_unique, df_starbux['docid'].is_unique

(True, True)

In [5]:
df_data = pd.concat([df_starbux, df_yht], axis=0)
df_data.reset_index(drop=True, inplace=True)
df_data

,docid,context,product_gpt,brand_gpt
0,bcf76d5a80f713fc1b28ceb62accffef,"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[]
1,bd003ce6a01a0fd0627f443a5cd9635a,别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[]
2,bd1ac8719255e993aa5ff2198c81c5aa,//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000D_#莫...,[抹茶星冰乐],[]
3,bd468468ea8a82bba99a718531c3b039,[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000D_打工人小葛喝...,[太妃榛果拿铁],[]
4,100fd688029bedd1b42de4ba13b0e9fd,[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000D_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克]
...,...,...,...,...
209376,2024010721779555610_1,"必须是新题材,新事物。\n周四我深水重仓中文在线,周五量化砸盘转盈为亏。在目前这个节点而言,...",[],"[中文在线, 鸿蒙, 东方中科, 亚华电子, 延华智能, 洲明科技, 光伏]"
209377,2024010721779693285_10,"年初至今,SW社会服务板块下跌1.14%,沪深300下跌2.97%,社会服务板块跑赢大盘1....","[70架飞机, 机场贵宾室, 免费机上Wi-Fi]","[SW社会服务, SW酒店餐饮, SW旅游及景区, SW教育, SW专业服务, 长白山, 大..."
209378,2024010721779853956_0,【01.07.解禁】美指小幅收涨！央行定调十项重点工作！国常会！数字经济促进共同富裕！证监会...,[],[]
209379,202401173863473_0,港股开盘 | 恒指低开0.64% 造车新势力、科网股表现较弱\n1月17日，恒生指数低开0....,"[恒生指数, 国企指数, 恒生科技指数, 道琼斯指数, 标普500指数, 纳斯达克指数, 美...","[蔚来, 小鹏汽车, 波音, AMD, 英伟达]"


## clean context

In [6]:
def clean_data(example):
    text = emoji.replace_emoji(example, replace="")
    text = OpenCC('t2s.json').convert(text)
    text = ''.join([x for x in text if x.isprintable()])
    text = text.lower()
    return text

pandarallel.initialize(nb_workers=32, progress_bar=True, use_memory_fs=False)
df_data["context_cleaned"] = df_data.parallel_apply(lambda x: clean_data(x["context"]), axis=1)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [7]:
df_data

,docid,context,product_gpt,brand_gpt,context_cleaned
0,bcf76d5a80f713fc1b28ceb62accffef,"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[],"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ..."
1,bd003ce6a01a0fd0627f443a5cd9635a,别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[],别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...
2,bd1ac8719255e993aa5ff2198c81c5aa,//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000D_#莫...,[抹茶星冰乐],[],//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000d_#莫...
3,bd468468ea8a82bba99a718531c3b039,[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000D_打工人小葛喝...,[太妃榛果拿铁],[],[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000d_打工人小葛喝...
4,100fd688029bedd1b42de4ba13b0e9fd,[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000D_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克],[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000d_隔着屏幕...
...,...,...,...,...,...
209376,2024010721779555610_1,"必须是新题材,新事物。\n周四我深水重仓中文在线,周五量化砸盘转盈为亏。在目前这个节点而言,...",[],"[中文在线, 鸿蒙, 东方中科, 亚华电子, 延华智能, 洲明科技, 光伏]","必须是新题材,新事物。周四我深水重仓中文在线,周五量化砸盘转盈为亏。在目前这个节点而言,中线..."
209377,2024010721779693285_10,"年初至今,SW社会服务板块下跌1.14%,沪深300下跌2.97%,社会服务板块跑赢大盘1....","[70架飞机, 机场贵宾室, 免费机上Wi-Fi]","[SW社会服务, SW酒店餐饮, SW旅游及景区, SW教育, SW专业服务, 长白山, 大...","年初至今,sw社会服务板块下跌1.14%,沪深300下跌2.97%,社会服务板块跑赢大盘1...."
209378,2024010721779853956_0,【01.07.解禁】美指小幅收涨！央行定调十项重点工作！国常会！数字经济促进共同富裕！证监会...,[],[],【01.07.解禁】美指小幅收涨！央行定调十项重点工作！国常会！数字经济促进共同富裕！证监会...
209379,202401173863473_0,港股开盘 | 恒指低开0.64% 造车新势力、科网股表现较弱\n1月17日，恒生指数低开0....,"[恒生指数, 国企指数, 恒生科技指数, 道琼斯指数, 标普500指数, 纳斯达克指数, 美...","[蔚来, 小鹏汽车, 波音, AMD, 英伟达]",港股开盘 | 恒指低开0.64% 造车新势力、科网股表现较弱1月17日，恒生指数低开0.64...


## clean label

In [8]:
brand_list_path = "../dict/brand_list_0314.txt" ### 改
product_list_path = "../dict/product_list_0314.txt" ### 改

with open(brand_list_path, "r") as f:
    brand_list = f.read().split("\n")

with open(product_list_path, "r") as f:
    product_list = f.read().split("\n")

In [9]:
len(brand_list), len(product_list)

(237, 8609)

In [10]:
tqdm.pandas()
df_data[['product_gt', 'brand_gt']] = df_data.progress_apply(lambda x: [
    [i for i in x['product_gpt'] if i in product_list],
    [i  for i in x['brand_gpt'] if i in brand_list]],
    axis=1).to_list()

  0%|          | 0/209381 [00:00<?, ?it/s]

/opt/conda/envs/ner/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


In [11]:
df_data

,docid,context,product_gpt,brand_gpt,context_cleaned,product_gt,brand_gt
0,bcf76d5a80f713fc1b28ceb62accffef,"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[],"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[]
1,bd003ce6a01a0fd0627f443a5cd9635a,别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[],别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[]
2,bd1ac8719255e993aa5ff2198c81c5aa,//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000D_#莫...,[抹茶星冰乐],[],//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000d_#莫...,[抹茶星冰乐],[]
3,bd468468ea8a82bba99a718531c3b039,[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000D_打工人小葛喝...,[太妃榛果拿铁],[],[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000d_打工人小葛喝...,[太妃榛果拿铁],[]
4,100fd688029bedd1b42de4ba13b0e9fd,[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000D_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克],[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000d_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克]
...,...,...,...,...,...,...,...
209376,2024010721779555610_1,"必须是新题材,新事物。\n周四我深水重仓中文在线,周五量化砸盘转盈为亏。在目前这个节点而言,...",[],"[中文在线, 鸿蒙, 东方中科, 亚华电子, 延华智能, 洲明科技, 光伏]","必须是新题材,新事物。周四我深水重仓中文在线,周五量化砸盘转盈为亏。在目前这个节点而言,中线...",[],[]
209377,2024010721779693285_10,"年初至今,SW社会服务板块下跌1.14%,沪深300下跌2.97%,社会服务板块跑赢大盘1....","[70架飞机, 机场贵宾室, 免费机上Wi-Fi]","[SW社会服务, SW酒店餐饮, SW旅游及景区, SW教育, SW专业服务, 长白山, 大...","年初至今,sw社会服务板块下跌1.14%,沪深300下跌2.97%,社会服务板块跑赢大盘1....",[],[]
209378,2024010721779853956_0,【01.07.解禁】美指小幅收涨！央行定调十项重点工作！国常会！数字经济促进共同富裕！证监会...,[],[],【01.07.解禁】美指小幅收涨！央行定调十项重点工作！国常会！数字经济促进共同富裕！证监会...,[],[]
209379,202401173863473_0,港股开盘 | 恒指低开0.64% 造车新势力、科网股表现较弱\n1月17日，恒生指数低开0....,"[恒生指数, 国企指数, 恒生科技指数, 道琼斯指数, 标普500指数, 纳斯达克指数, 美...","[蔚来, 小鹏汽车, 波音, AMD, 英伟达]",港股开盘 | 恒指低开0.64% 造车新势力、科网股表现较弱1月17日，恒生指数低开0.64...,[],[]


## keyword list 回扫

In [12]:
def entend_label_and_merge_content_keyword(example, threshold=-1.0):
    brand_labels = example['brand_gt'].copy()
    product_labels = example['product_gt'].copy()
    text = example['context_cleaned']
    brand_matched_keywords, product_matched_keywords = [], []
    if pd.notna(text):
        for keyword in brand_list:
            if keyword.lower() in text.lower():
                brand_matched_keywords.append(keyword.lower())
        for keyword in product_list:
            if keyword.lower() in text.lower():
                product_matched_keywords.append(keyword.lower())

    brand_labels.extend(brand_matched_keywords)
    product_labels.extend(product_matched_keywords)
    brand_labels = list(set(brand_labels))
    product_labels = list(set(product_labels))
    brand_labels_, product_labels_ = [], []
    for i in brand_labels:
        temp = brand_labels.copy()
        temp.remove(i)
        flag = 1
        for j in temp:
            if i not in j:
                continue
            else:
                flag = 0
                break
        if flag:
            brand_labels_.append(i)
    for i in product_labels:
        temp = product_labels.copy()
        temp.remove(i)
        flag = 1
        for j in temp:
            if i not in j:
                continue
            else:
                flag = 0
                break
        if flag:
            product_labels_.append(i)
    
    labels_ = brand_labels_ + product_labels_
    

    text_list = []
    separator = "。！？#?!"
    start, end = 0, 0
    while end < len(text):
        if text[end] in separator:
            text_list.append(text[start:end+1])
            start, end = end + 1, end + 1
        else:
            end += 1
        
        if end == len(text) - 1:
            text_list.append(text[start:end+1])

    for idx, t in enumerate(text_list):
        temp = []
        for l in labels_:
            if l in t:
                temp.append(l)
        if temp != []:
            text_list[idx] = text_list[idx] + '||可能的实体：' + ",".join(temp) + '||' 
    return product_labels_, brand_labels_, labels_, "".join(text_list)

In [13]:
df_data[["product_matched", "brand_matched", "matched_keywords", "context_keywords"]] = df_data.parallel_apply(entend_label_and_merge_content_keyword, axis=1).to_list()
df_data

/opt/conda/envs/ner/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


,docid,context,product_gpt,brand_gpt,context_cleaned,product_gt,brand_gt,product_matched,brand_matched,matched_keywords,context_keywords
0,bcf76d5a80f713fc1b28ceb62accffef,"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[],"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[],[抹茶星冰乐],[],[抹茶星冰乐],"//@抹茶星冰乐mi:#||可能的实体：抹茶星冰乐||创维电视 足够精彩# 眼中有美,心中有..."
1,bd003ce6a01a0fd0627f443a5cd9635a,别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[],别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[],[摩卡星冰乐],[],[摩卡星冰乐],别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...
2,bd1ac8719255e993aa5ff2198c81c5aa,//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000D_#莫...,[抹茶星冰乐],[],//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000d_#莫...,[抹茶星冰乐],[],[抹茶星冰乐],[],[抹茶星冰乐],//@贺一杯抹茶星冰乐200502:。||可能的实体：抹茶星冰乐||//@贺一杯抹茶星冰乐2...
3,bd468468ea8a82bba99a718531c3b039,[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000D_打工人小葛喝...,[太妃榛果拿铁],[],[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000d_打工人小葛喝...,[太妃榛果拿铁],[],"[太妃榛果拿铁, 玉米汁, 芋泥脑袋, 鸭屎香, 生牛乳]",[星爸爸],"[星爸爸, 太妃榛果拿铁, 玉米汁, 芋泥脑袋, 鸭屎香, 生牛乳]",[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000d_打工人小葛喝...
4,100fd688029bedd1b42de4ba13b0e9fd,[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000D_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克],[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000d_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克],"[阿拉比卡, pink drink粉粉生咖, dragon drink幻紫生咖, 咖啡生豆,...",[星巴克],"[星巴克, 阿拉比卡, pink drink粉粉生咖, dragon drink幻紫生咖, ...",[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000d_隔着屏幕...
...,...,...,...,...,...,...,...,...,...,...,...
209376,2024010721779555610_1,"必须是新题材,新事物。\n周四我深水重仓中文在线,周五量化砸盘转盈为亏。在目前这个节点而言,...",[],"[中文在线, 鸿蒙, 东方中科, 亚华电子, 延华智能, 洲明科技, 光伏]","必须是新题材,新事物。周四我深水重仓中文在线,周五量化砸盘转盈为亏。在目前这个节点而言,中线...",[],[],[],[],[],"必须是新题材,新事物。周四我深水重仓中文在线,周五量化砸盘转盈为亏。在目前这个节点而言,中线..."
209377,2024010721779693285_10,"年初至今,SW社会服务板块下跌1.14%,沪深300下跌2.97%,社会服务板块跑赢大盘1....","[70架飞机, 机场贵宾室, 免费机上Wi-Fi]","[SW社会服务, SW酒店餐饮, SW旅游及景区, SW教育, SW专业服务, 长白山, 大...","年初至今,sw社会服务板块下跌1.14%,沪深300下跌2.97%,社会服务板块跑赢大盘1....",[],[],[],[],[],"年初至今,sw社会服务板块下跌1.14%,沪深300下跌2.97%,社会服务板块跑赢大盘1...."
209378,2024010721779853956_0,【01.07.解禁】美指小幅收涨！央行定调十项重点工作！国常会！数字经济促进共同富裕！证监会...,[],[],【01.07.解禁】美指小幅收涨！央行定调十项重点工作！国常会！数字经济促进共同富裕！证监会...,[],[],[],[],[],【01.07.解禁】美指小幅收涨！央行定调十项重点工作！国常会！数字经济促进共同富裕！证监会...
209379,202401173863473_0,港股开盘 | 恒指低开0.64% 造车新势力、科网股表现较弱\n1月17日，恒生指数低开0....,"[恒生指数, 国企指数, 恒生科技指数, 道琼斯指数, 标普500指数, 纳斯达克指数, 美...","[蔚来, 小鹏汽车, 波音, AMD, 英伟达]",港股开盘 | 恒指低开0.64% 造车新势力、科网股表现较弱1月17日，恒生指数低开0.64...,[],[],[],[],[],港股开盘 | 恒指低开0.64% 造车新势力、科网股表现较弱1月17日，恒生指数低开0.64...


In [14]:
count = 0
empty_index_list = []
for index, i in enumerate(zip(df_data.loc[157851:, 'product_matched'], df_data.loc[157851:, 'brand_matched'])):
    if i[0] == [] and i[1] == []:
        count += 1
        empty_index_list.append(index+157851)
count, len(empty_index_list)

(19446, 19446)

In [15]:
count = 0
for i, j in zip(df_data['product_matched'], df_data['brand_matched']):
    if i == [] and j == []:
        count += 1
count

24025

In [16]:
count = 0
for i in df_data.loc[157851:, 'product_matched']:
    if i == []:
        count += 1
count

35396

In [17]:
count = 0
for i in df_data['product_matched']:
    if i == []:
        count += 1
count

115491

In [18]:
seed = 42
remove_index_list = random.sample(empty_index_list, 17000)
len(remove_index_list)

17000

In [19]:
df_data = df_data.drop(index=remove_index_list)
df_data.reset_index(drop=True, inplace=True)
df_data

,docid,context,product_gpt,brand_gpt,context_cleaned,product_gt,brand_gt,product_matched,brand_matched,matched_keywords,context_keywords
0,bcf76d5a80f713fc1b28ceb62accffef,"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[],"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[],[抹茶星冰乐],[],[抹茶星冰乐],"//@抹茶星冰乐mi:#||可能的实体：抹茶星冰乐||创维电视 足够精彩# 眼中有美,心中有..."
1,bd003ce6a01a0fd0627f443a5cd9635a,别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[],别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[],[摩卡星冰乐],[],[摩卡星冰乐],别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...
2,bd1ac8719255e993aa5ff2198c81c5aa,//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000D_#莫...,[抹茶星冰乐],[],//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000d_#莫...,[抹茶星冰乐],[],[抹茶星冰乐],[],[抹茶星冰乐],//@贺一杯抹茶星冰乐200502:。||可能的实体：抹茶星冰乐||//@贺一杯抹茶星冰乐2...
3,bd468468ea8a82bba99a718531c3b039,[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000D_打工人小葛喝...,[太妃榛果拿铁],[],[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000d_打工人小葛喝...,[太妃榛果拿铁],[],"[太妃榛果拿铁, 玉米汁, 芋泥脑袋, 鸭屎香, 生牛乳]",[星爸爸],"[星爸爸, 太妃榛果拿铁, 玉米汁, 芋泥脑袋, 鸭屎香, 生牛乳]",[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000d_打工人小葛喝...
4,100fd688029bedd1b42de4ba13b0e9fd,[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000D_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克],[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000d_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克],"[阿拉比卡, pink drink粉粉生咖, dragon drink幻紫生咖, 咖啡生豆,...",[星巴克],"[星巴克, 阿拉比卡, pink drink粉粉生咖, dragon drink幻紫生咖, ...",[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000d_隔着屏幕...
...,...,...,...,...,...,...,...,...,...,...,...
192376,2024010521789651196_1,"23年上市的新股中，""周杰伦""概念股$巨星传奇(06683.HK)$，全年收涨77%；$乐华...","[06683.HK, 02306.HK, 02373.HK, 02405.HK, 09890...","[巨星传奇, 乐华娱乐, 美丽田园医疗健康, 力盟科技, 贪玩游戏运营商中旭未来, 药明合联]","23年上市的新股中，""周杰伦""概念股$巨星传奇(06683.hk)$，全年收涨77%；$乐华...",[],[],[],[],[],"23年上市的新股中，""周杰伦""概念股$巨星传奇(06683.hk)$，全年收涨77%；$乐华..."
192377,202401176642303_0,【商界头条】华为承诺不从事整车业务;乐华娱乐大跌超70%;蜜雪冰城回应暂停瓶装饮料\n商界头...,"[瓶装饮料, 华为智能汽车解决方案BU, 乐华娱乐管理服务]","[华为, 蜜雪冰城, 乐华娱乐]",【商界头条】华为承诺不从事整车业务;乐华娱乐大跌超70%;蜜雪冰城回应暂停瓶装饮料商界头条丨...,[],[蜜雪冰城],[],[蜜雪冰城],[蜜雪冰城],【商界头条】华为承诺不从事整车业务;乐华娱乐大跌超70%;蜜雪冰城回应暂停瓶装饮料商界头条丨...
192378,2024010121777603343_4,"板块涨幅方面:调味发酵品(+6.63%)涨幅居前,其他食品(0.36%)涨幅较小。个股涨跌方...","[调味发酵品, 其他食品]","[莲花健康, 香飘飘, 华康股份, 威龙股份]","板块涨幅方面:调味发酵品(+6.63%)涨幅居前,其他食品(0.36%)涨幅较小。个股涨跌方...",[],[香飘飘],[],[香飘飘],[香飘飘],"板块涨幅方面:调味发酵品(+6.63%)涨幅居前,其他食品(0.36%)涨幅较小。个股涨跌方..."
192379,202401173279344_0,投资界24h | 乐华回应股价暴跌80%;高盛Q4净营收113.2亿美元;福建100亿元政府...,[瓶装饮料项目],"[乐华娱乐集团, 蜜雪冰城]",投资界24h | 乐华回应股价暴跌80%;高盛q4净营收113.2亿美元;福建100亿元政府...,[],[蜜雪冰城],[],[蜜雪冰城],[蜜雪冰城],投资界24h | 乐华回应股价暴跌80%;高盛q4净营收113.2亿美元;福建100亿元政府...


In [20]:
df_data.to_pickle("data_starbucks_yihetang_aligned.pkl")

## ...不用管

In [24]:
for _, i in enumerate(df_data['context_cleaned']):
    if "starbucks" in i:
        print(_)

28
55
72
173
188
202
217
391
444
519
523
585
616
640
719
1033
1145
1420
1476
1576
1641
1698
1780
1859
1970
1974
2010
4949
4965
4992
5004
5008
5013
5026
5093
5140
5289
5305
5352
5449
5503
5531
5558
5559
5587
5637
5723
5901
6091
6157
6292
6294
6310
6318
6324
6373
6389
17601
17859
18021
18415
19972
20006
20618
21162
21204
21262
21523
22537
22681
23821
23889
24091
25343
25685
26874
28146
28282
29794
30508
31681
31886
31958
33208
34829
34941
37168
37849
39226
39756
39767
39943
40554
40623
40630
40758
40796
40806
40815
40819
40837
40845
40858
40883
40913
40931
40955
40972
40980
40996
40997
41002
41008
41011
41015
41023
41070
41076
41095
41110
41112
41116
41119
41132
41145
41158
41182
41190
41191
41194
41216
41226
41248
41261
41270
41281
41329
41344
41348
41356
41371
41373
41451
41457
41486
41492
41503
41534
41545
41562
41566
41584
41591
41598
41616
41624
41673
41678
41765
41768
41778
41783
41840
41842
41850
41907
41917
41926
41931
41942
41944
41967
41970
41973
41988
42013
42020
42025
42047
4

In [29]:
df_data.iloc[149103]

docid                                ffb5cf4925bb80ef2dfac8ae7130dda1
context             星巴克vs喜茶，你们pick谁？ 1. 受众: 星巴克(Starbucks): 星巴克主要受...
product_gpt                                                        []
brand_gpt                                                          []
context_cleaned     星巴克vs喜茶，你们pick谁？ 1. 受众: 星巴克(starbucks): 星巴克主要受...
product_gt                                                         []
brand_gt                                                           []
product_matched                                                 [咖啡豆]
brand_matched                            [星巴克, 喜茶, starbucks, heytea]
matched_keywords                    [星巴克, 喜茶, starbucks, heytea, 咖啡豆]
context_keywords    星巴克vs喜茶，你们pick谁？||可能的实体：星巴克,喜茶|| 1. 受众: 星巴克(st...
Name: 149103, dtype: object

In [5]:
import pandas as pd

df_json = pd.read_pickle("data_starbucks_yihetang_aligned.pkl")

In [6]:
df_json.to_json("data_starbucks_yihetang_aligned.json",orient="records")

In [7]:
pd.read_json("data_starbucks_yihetang_aligned.json")

,docid,context,product_gpt,brand_gpt,context_cleaned,product_gt,brand_gt,product_matched,brand_matched,matched_keywords,context_keywords
0,bcf76d5a80f713fc1b28ceb62accffef,"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[],"//@抹茶星冰乐mi:#创维电视 足够精彩# 眼中有美,心中有爱,每一天都是春暖花开,早安 ...",[抹茶星冰乐],[],[抹茶星冰乐],[],[抹茶星冰乐],"//@抹茶星冰乐mi:#||可能的实体：抹茶星冰乐||创维电视 足够精彩# 眼中有美,心中有..."
1,bd003ce6a01a0fd0627f443a5cd9635a,别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[],别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...,[],[],[摩卡星冰乐],[],[摩卡星冰乐],别一说完就拉黑啊?在网络上发言都这样经不起辩论吗?[doge]//@乐林1982:回复@阴阳...
2,bd1ac8719255e993aa5ff2198c81c5aa,//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000D_#莫...,[抹茶星冰乐],[],//@贺一杯抹茶星冰乐200502:。//@贺一杯抹茶星冰乐200502:_x000d_#莫...,[抹茶星冰乐],[],[抹茶星冰乐],[],[抹茶星冰乐],//@贺一杯抹茶星冰乐200502:。||可能的实体：抹茶星冰乐||//@贺一杯抹茶星冰乐2...
3,bd468468ea8a82bba99a718531c3b039,[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000D_打工人小葛喝...,[太妃榛果拿铁],[],[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000d_打工人小葛喝...,[太妃榛果拿铁],[],"[太妃榛果拿铁, 玉米汁, 芋泥脑袋, 鸭屎香, 生牛乳]",[星爸爸],"[星爸爸, 太妃榛果拿铁, 玉米汁, 芋泥脑袋, 鸭屎香, 生牛乳]",[舔屏]//@璃汐in:转发微博。[舔屏]//@璃汐in:转发微博_x000d_打工人小葛喝...
4,100fd688029bedd1b42de4ba13b0e9fd,[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000D_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克],[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000d_隔着屏幕...,"[粉粉生咖, 幻紫生咖, 粉莓柠力生咖, 芒紫柠力生咖]",[星巴克],"[阿拉比卡, pink drink粉粉生咖, dragon drink幻紫生咖, 咖啡生豆,...",[星巴克],"[星巴克, 阿拉比卡, pink drink粉粉生咖, dragon drink幻紫生咖, ...",[挤眼]//@请输入昵称ya-:5。[挤眼]//@请输入昵称ya-:5_x000d_隔着屏幕...
...,...,...,...,...,...,...,...,...,...,...,...
192376,2024010521789651196_1,"23年上市的新股中，""周杰伦""概念股$巨星传奇(06683.HK)$，全年收涨77%；$乐华...","[06683.HK, 02306.HK, 02373.HK, 02405.HK, 09890...","[巨星传奇, 乐华娱乐, 美丽田园医疗健康, 力盟科技, 贪玩游戏运营商中旭未来, 药明合联]","23年上市的新股中，""周杰伦""概念股$巨星传奇(06683.hk)$，全年收涨77%；$乐华...",[],[],[],[],[],"23年上市的新股中，""周杰伦""概念股$巨星传奇(06683.hk)$，全年收涨77%；$乐华..."
192377,202401176642303_0,【商界头条】华为承诺不从事整车业务;乐华娱乐大跌超70%;蜜雪冰城回应暂停瓶装饮料\n商界头...,"[瓶装饮料, 华为智能汽车解决方案BU, 乐华娱乐管理服务]","[华为, 蜜雪冰城, 乐华娱乐]",【商界头条】华为承诺不从事整车业务;乐华娱乐大跌超70%;蜜雪冰城回应暂停瓶装饮料商界头条丨...,[],[蜜雪冰城],[],[蜜雪冰城],[蜜雪冰城],【商界头条】华为承诺不从事整车业务;乐华娱乐大跌超70%;蜜雪冰城回应暂停瓶装饮料商界头条丨...
192378,2024010121777603343_4,"板块涨幅方面:调味发酵品(+6.63%)涨幅居前,其他食品(0.36%)涨幅较小。个股涨跌方...","[调味发酵品, 其他食品]","[莲花健康, 香飘飘, 华康股份, 威龙股份]","板块涨幅方面:调味发酵品(+6.63%)涨幅居前,其他食品(0.36%)涨幅较小。个股涨跌方...",[],[香飘飘],[],[香飘飘],[香飘飘],"板块涨幅方面:调味发酵品(+6.63%)涨幅居前,其他食品(0.36%)涨幅较小。个股涨跌方..."
192379,202401173279344_0,投资界24h | 乐华回应股价暴跌80%;高盛Q4净营收113.2亿美元;福建100亿元政府...,[瓶装饮料项目],"[乐华娱乐集团, 蜜雪冰城]",投资界24h | 乐华回应股价暴跌80%;高盛q4净营收113.2亿美元;福建100亿元政府...,[],[蜜雪冰城],[],[蜜雪冰城],[蜜雪冰城],投资界24h | 乐华回应股价暴跌80%;高盛q4净营收113.2亿美元;福建100亿元政府...
